In [1]:
import hvplot.xarray  # noqa
import xarray as xr
import numpy as np
import panel as pn
import panel.widgets as pnw

# For performance reasons sliders should only re-evaluate on mouseup
pn.config.throttled = True

## Bootstrap samples (bootstrap_mean, bootstrap_median, bootstrap_std) and full counterfactual (cfact)

Note: You might need to generate the data first, e.g. by calling
```
attrici detrend \
    --gmt-file tests/data/20CRv3-ERA5_germany_ssa_gmt.nc \
    --input-file tests/data/20CRv3-ERA5_germany_obs.nc \
    --variable tas \
    --output-dir tests/data/output \
    --bootstrap-sample-count 100 \
    --overwrite
```

In [2]:
ds = xr.open_mfdataset("../tests/data/output/bootstrap/tas/**/*.nc")
ds

<xarray.Dataset> Size: 6MB
Dimensions:           (lon: 2, lat: 2, time: 44925)
Coordinates:
  * time              (time) datetime64[ns] 359kB 1901-01-01 ... 2023-12-31
  * lon               (lon) float64 16B 9.25 9.75
  * lat               (lat) float64 16B 50.75 51.25
Data variables:
    bootstrap_std     (lon, lat, time) float64 1MB dask.array<chunksize=(1, 1, 44925), meta=np.ndarray>
    bootstrap_mean    (lon, lat, time) float64 1MB dask.array<chunksize=(1, 1, 44925), meta=np.ndarray>
    bootstrap_median  (lon, lat, time) float64 1MB dask.array<chunksize=(1, 1, 44925), meta=np.ndarray>
    cfact             (lon, lat, time) float64 1MB dask.array<chunksize=(1, 1, 44925), meta=np.ndarray>
Attributes:
    attrici_version:         1.1.1.dev128+g7244946
    attrici_packages:        anyio==4.8.0\nargon2-cffi-bindings==21.2.0\nargo...
    attrici_python_version:  3.12.7 (main, Nov  8 2024, 17:55:36) [GCC 14.2.0]
    created_at:              2025-01-22T10:57:56.658940
    attrici_config:          gmt_file = "tests/data/20CRv3-ERA5_germany_ssa_g...

In [3]:
def plot(d, **kwargs):
    lon_range = d.lon.max() - d.lon.min()
    lat_range = d.lat.max() - d.lat.min()
    lon_margin = 7 * (lon_range - 360) / (0.5 - 360)
    lat_margin = 7 * (lat_range - 360) / (0.5 - 360)
    return d.hvplot(
        "lon",
        "lat",
        xlim=(d.lon.min() - lon_margin, d.lon.max() + lon_margin),
        ylim=(d.lat.min() - lat_margin, d.lat.max() + lat_margin),
        geo=True,
        coastline=True,
        **kwargs,
    )

## Example plot of bootstrap mean

In [4]:
plot(ds.bootstrap_mean.isel(time=-1), title="Bootstrap Mean (last timestep)")

:Overlay
   .Image.I     :Image   [lon,lat]   (Near-Surface Air Temperature)
   .Coastline.I :Feature   [Longitude,Latitude]

## Bootstrap Mean (with date selection slider)

In [5]:
plot(
    ds.bootstrap_mean,
    groupby="time",
    widget_location="top",
    title="Bootstrap Mean",
)

Column
    [0] WidgetBox(align=('center', 'start'))
        [0] DiscreteSlider(name='time', options={'1901-01-01 00:00:00': nu...}, value=numpy.datetime64('1901-01-...)
    [1] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='top')

## RMSE of bootstrap mean and counterfactual for last year

In [6]:
def rmse(d1, d2, time=None, interactive=False):
    if time is None:
        # use last year by default
        time = str(list(d1.time.groupby("time.year").groups.keys())[-1])
    diff = d1 - d2
    if interactive:
        # if time is interactive, interactive() has to be called inside this
        # function, before mean(dim="time"). also, the difference needs a name.
        diff = diff.rename("difference").interactive()
    return np.sqrt((diff.sel(time=time) ** 2).mean(dim="time"))

In [7]:
plot(rmse(ds.cfact, ds.bootstrap_mean), title="RMSE (cfact, bootstrap_mean) last year")

:Overlay
   .Image.I     :Image   [lon,lat]   (value)
   .Coastline.I :Feature   [Longitude,Latitude]

## Annual Mean of Bootstrap Standard Deviation

In [8]:
selected_year = "2023"
plot(
    ds.bootstrap_std.sel(time=selected_year).mean(dim="time"),
    title=f"Mean of Boostrap Std {selected_year}",
)

:Overlay
   .Image.I     :Image   [lon,lat]   (bootstrap_std)
   .Coastline.I :Feature   [Longitude,Latitude]

## With interactive year selection

In [9]:
years = list(ds.time.groupby("time.year").groups.keys())
year = pnw.DiscreteSlider(
    name="time", options=[str(i) for i in years], value=str(years[-1])
)

rmse_plot = plot(
    rmse(ds.cfact, ds.bootstrap_mean, time=year, interactive=True),
    title="Annual RMSE (cfact, bootstrap_mean)",
)
mean_bs_std_plot = plot(
    ds.bootstrap_std.interactive().sel(time=year).mean(dim="time"),
    title="Annual mean of Bootstrap Std",
)

pn.Row(rmse_plot, mean_bs_std_plot)

Row
    [0] Interactive(XArrayInteractive)
    [1] Interactive(XArrayInteractive)